In [85]:
from pathlib import Path
import pandas as pd
import json
# import shap

In [86]:
PATH = Path(r'C:\Users\niti.mishra\Documents\2_TDMDAL\covid_tests\covid_tests')
# PATH = Path(r'/Users/peaceforlives/Documents/Projects/covid_symptoms')
DATA = Path.joinpath(PATH, 'data')
RESULTS = Path.joinpath(PATH, 'results')

## DATA ##

In [87]:
covid_data = pd.read_csv(Path.joinpath(DATA, 'RawData_IMOH_Sep_20.csv'))

In [88]:
x = covid_data[["cough","fever","sore_throat","shortness_of_breath", "head_ache"]]

In [89]:
y = covid_data['testresult']
cols = ['cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache', 'sixtiesplus', 'Gender', 'contact', 'abroad', 'Validation']
covid_data = covid_data[cols]

In [90]:
covid_data.index = covid_data['Validation']
covid_data.drop('Validation', inplace=True, axis=1)
covid_data.head()

,cough,fever,sore_throat,shortness_of_breath,head_ache,sixtiesplus,Gender,contact,abroad
Validation,,,,,,,,,
Training,0,1,0,0,0,1,1,0,0
Validation,0,1,0,0,0,0,0,0,0
Test,1,0,0,0,1,0,1,0,0
Test,1,0,0,0,0,0,0,1,0
Training,0,0,0,0,1,0,1,0,0


In [91]:
train = covid_data.index.get_loc('Training')
valid = covid_data.index.get_loc('Validation')
test = covid_data.index.get_loc('Test')

In [98]:
covid_data.shape

(82919, 9)

In [92]:
X_train = covid_data[train].values.tolist()
y_train = y[train].values.tolist()
print(len(y_train))

X_valid = covid_data[valid].values.tolist()
y_valid = y[valid].values.tolist()
print(len(y_valid))

X_test = covid_data[test].values.tolist()
y_test = y[test].values.tolist()
print(len(y_test))

41464
20744
20711


In [32]:
import numpy as np
import time

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, auc, roc_auc_score


In [76]:
binary_models = []
binary_models.append( LogisticRegression(random_state = 42) )
# binary_models.append( RandomForestClassifier(random_state = 42) )

parameters = [
    {'clf__C': ( np.logspace(-5, 1, 5) ),
        'clf__penalty': ['l1', 'l2', 'none'] # regularization paramter
        },
    {'clf__max_depth': [int(x) for x in np.linspace(10, 110, num = 11)],
        'clf__min_samples_split': [2, 5, 10],
        'clf__min_samples_leaf': [1, 2, 4]
        },
    {'clf__C': [0.001, 0.01, 0.1, 1, 10],
        'clf__gamma': [0.001, 0.01, 0.1, 1]
        }
]

In [78]:
models = binary_models

names = [str(i).split('(')[0] for i in models]
for model, name, params in zip(models, names, parameters):
    print(model, '\n', name, '\n', params)  

    pipe = Pipeline([
        ('clf', model),
        ])

    # Create a list where train data indices are -1 and validation data indices are 0
    test_fold = [-1 for i in range(n_Xtrn)] + [0 for i in range(n_Xval)]
    ps = PredefinedSplit(test_fold=test_fold)

    # Create gridsearch with specified valid set
    grid_search = GridSearchCV(estimator=pipe, param_grid=params, cv=ps)#, scoring='roc_auc')
    start = time.time()
    best_model = grid_search.fit(X_train+X_valid, y_train+y_valid)
    best_param = best_model.best_params_
    best_score = best_model.best_score_
    best_estimator = best_model.best_estimator_['clf']
    print(best_param)
    print(best_score)
    print(best_estimator)

    if name=='LogisticRegression':
        coef = np.concatenate((best_estimator.intercept_ , best_estimator.coef_[0]))
    else:
        coef = list([None])

    y_pred = best_model.predict(X_test)
    y_valid_pred = best_model.predict(X_valid)
    y_pred_prob = best_model.predict_proba(X_test)
    y_pred_l = [str(i) for i in y_pred]
    print(type(y_test[0]))
    print(type(y_test))
    print(type(y_pred[0]))
    print(type(y_pred))
    print(type(y_pred_l[0]))
    print(type(y_pred_l))
    # save results
    scores = {
        'time': time.time() - start,
        'name': name,
        'model': str(best_estimator),
        'size': [len(X_train), len(X_valid), len(X_test)],

        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, average='weighted'),
        'recall': recall_score(y_test, y_pred, average='weighted'),
        'auc': roc_auc_score([int(i)for i in y_test], [int(i)for i in y_pred_l], average='weighted'),
        'f1_test': f1_score(y_test, y_pred, average='weighted'),
        'f1_valid': f1_score(y_valid, y_valid_pred, average='weighted'),
        'coef': list(coef),
    }
    print('model: ', scores['name'])
    print('accuracy: ', scores['accuracy'])
    print('precision: ', scores['precision'])
    print('recall: ', scores['recall'])
    print('f1_test: ', scores['f1_test'])
    print('f1_valid: ', scores['f1_valid'])
    print('auc: ', scores['auc'])
    print('coef: ', scores['coef'])
    print('time: ', scores['time'])

LogisticRegression(random_state=42) 
 LogisticRegression 
 {'clf__C': array([1.00000000e-05, 3.16227766e-04, 1.00000000e-02, 3.16227766e-01,
       1.00000000e+01]), 'clf__penalty': ['l1', 'l2', 'none']}


C:\Users\niti.mishra\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\niti.mishra\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\niti.mishra\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\niti.mishra\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\niti.mishra\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 443, in _check_

{'clf__C': 0.00031622776601683794, 'clf__penalty': 'l2'}
0.7866370998843039
LogisticRegression(C=0.00031622776601683794, random_state=42)
<class 'int'>
<class 'list'>
<class 'numpy.int32'>
<class 'numpy.ndarray'>
<class 'str'>
<class 'list'>
model:  LogisticRegression
accuracy:  0.7908840712664768
precision:  0.7939437128652741
recall:  0.7908840712664768
f1_test:  0.7871425346924498
f1_valid:  0.7838687630616331
auc:  0.7759486987562624
coef:  [0.24516100402530813, -0.5511478335319575, 0.22916253119783064, 0.326313639689523, 0.19066297891663683, 0.598398070063963, 0.07997134526608221, -0.01469578741190504, 0.46338877809469886, -0.7444467897651866]
time:  1.6476173400878906


In [64]:
best_model.best_estimator_['clf'] 

RandomForestClassifier(max_depth=10, min_samples_leaf=4, min_samples_split=10,
                       random_state=42)

In [87]:
grid_search.best_estimator_['clf'].coef_[0]
# grid_search.best_estimator_['clf'].intercept_

array([-0.55114783,  0.22916253,  0.32631364,  0.19066298,  0.59839807,
        0.07997135, -0.01469579,  0.46338878, -0.74444679])

In [80]:
from collections import Counter
Counter(y_test)

Counter({1: 11765, 0: 8946})

In [82]:
Counter(y_pred)

Counter({1: 13406, 0: 7305})

In [84]:
Counter([int(i) for i in y_pred])

Counter({1: 13406, 0: 7305})

In [68]:
type(y_test[0])

int

In [71]:
roc_auc_score(y_test, y_pred_l)

0.7787186451570546

In [20]:
# # rename columns
# cols = ['test_date']
# cols.extend(covid_data.columns[1:] )
# covid_data.columns = cols

# # select columns
# cols = ['test_date', 'cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache',
#         'testresult', 'sixtiesplus', 'Gender', 'contact', 'abroad', 'Validation']
# covid_data = covid_data[cols]

# covid_data['test_date'] = pd.to_datetime(covid_data['test_date'], format='%Y-%m-%d')

# covid_data.head()

## RESULTS ##

In [99]:
res = pd.read_json(str(RESULTS)+'/results.json', orient='records', lines=True)

In [102]:
res.set_index('time', inplace=True)

In [103]:
res.to_clipboard()

In [33]:
import shap

ModuleNotFoundError: No module named 'shap'